In [2]:
pip install newspaper3k

  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\81\2b\43\a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\83\15\9c\a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\de\03\ca\778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
Successfully built tinysegmenter jieba3k feedfinder2
Note: you may need to restart the kernel to use updated packages.


In [5]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [35]:
#크롤링할 url 주소 입력
url = 'http://www.asiatoday.co.kr/view.php?key=20190802000725171'
#언어가 한국어이므로 language='ko'로 설정
a = Article(url, language='ko')
a.download()
a.parse()
#기사 제목 가져오기
print(a.title)
#기사 내용 가져오기(150자)
print(a.text[:150])

청년 내일채움공제, 중소·중견기업 근속 청년 공제 사업…신청방법은?
0 청년 내일채움공제에 관심이 쏠리고 있다. /청년 내일채움공제 홈페이지 아시아투데이 서현정 기자 = 2일 '청년 내일채움공제'가 주목받고 있다. 아시아투데이 서현정 기자 = 2일 '청년 내일채움공제'가 주목받고 있다.

'청년 내일채움공제'란 중소·중견기업에 정규직으


In [36]:
a.text

"0 청년 내일채움공제에 관심이 쏠리고 있다. /청년 내일채움공제 홈페이지 아시아투데이 서현정 기자 = 2일 '청년 내일채움공제'가 주목받고 있다. 아시아투데이 서현정 기자 = 2일 '청년 내일채움공제'가 주목받고 있다.\n\n'청년 내일채움공제'란 중소·중견기업에 정규직으로 취업한 청년들의 장기근속을 위해 고용노동부와 중소벤처기업부가 공동으로 운영하는 정책성 공제사업이다.\n\n사업은 청년·기업·정부가 공동으로 공제금을 적립하여 2년 또는 3년간 근속한 청년에게 성과보상금 형태로 만기공제금을 지급하는 식으로 운영된다.\n\n청년 가입조건은 정규직 취업일 나이 만 15세 이상 34세 이하 해당 자이며, 정규직 취업일 현재 고용보험 이력이 없거나 최종학교 졸업 후 이전 직장까지의 고용보험 총 가입기간이 12개월 이하인 자이다.\n\n고등학교 및 대학 재학 중인 자, 청년공제에 가입했던 자, 월 급여총액이 500만원 초과한 자, 소정근로시간이 주 30시간 미만인 자, 재택근무자 등은 가입이 제한된다.\n\n기업 가입조건은 청년 공제 가입 대상인 청년의 정규직 채용일 기준 고용보험 피보험자 수 5인 이상인 '중소기업기본법' 상 중소기업 또는 '중견기업 성장촉진 및 경쟁력 강화에 관한 법률' 상 중견기업에 해당하는 기업이다. (다만 벤처기업, 청년 창업기업 등 일부는 1인 이상 5인 미만 기업도 가능하다.)\n\n\n\n신청은 청년내일채움공제 홈페이지에서 가능하며, 정규직 취업일(채용일)로부터 3개월 이내에 이뤄져야 한다.\n\n이외 관련 자세한 사항은 청년 내일채움공제 공식 홈페이지에서 확인 할 수 있다.\n\n"

In [37]:
#Step 1. 문서 타입에 따른 문장 단위로 분리하기(텍스트 크롤링 > 문장 단위 분리 > 명사 추출)
#<SentenceTokenizer Class>
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Twitter()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns


In [38]:
#Step 2. TF-IDF 모델 생성 및 그래프 생성
#<GrpahMatrix Class>

class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
        
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [39]:
#Step 4. TextRank Class 구현
#<Rank class>
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
                A[:, id] *= -d
                A[id, id] = 1
        
        
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}




In [40]:
#Step 4. TextRank Class 구현

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)

            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary

    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        
        
        #index.sort()
        
        for idx in index:
            keywords.append(self.idx2word[idx])
        
        return keywords


In [43]:
# Step 5. 결과 확인

url = 'http://www.asiatoday.co.kr/view.php?key=20190802000725171'

textrank = TextRank(url)

for row in textrank.summarize(5):
    print(row)
    print()


print('keywords :',textrank.keywords())


C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


/ 청년 내일채 움 공제 홈페이지 아시아 투데이 서 현 정 기자 = 2일 ' 청년 내일채 움 공제' 가 주목받고 있다.

아시아 투데이 서 현 정 기자 = 2일 ' 청년 내일채 움 공제' 가 주목받고 있다.

' 청년 내일채 움 공제' 란 중소· 중견기업에 정규직으로 취업한 청년들의 장기 근속을 위해 고용 노동부와 중소 벤처기업 부가 공동으로 운영하는 정책성 공제사업이다.

기업 가입조건은 청년 공제 가입 대상인 청년의 정규직 채용 일 기준 고용보험 피보험자 수 5 인 이상인 ' 중소기업 기본법' 상 중소기업 또는 ' 중견기업 성장 촉진 및 경쟁력 강화에 관한 법률' 상 중견기업에 해당하는 기업이다.

신청은 청년 내일채 움 공제 홈페이지에서 가능하며, 정규직 취업 일( 채용 일 )로부터 3개월 이내에 이뤄 져야 한다.

keywords : ['청년', '가입', '공제', '정규직', '고용', '기업', '보험', '조건', '해당', '취업']
